In [30]:
from agent.agent import Agent
import pandas as pd
import numpy as np
from custombt.backtest import Backtest
from custombt.strategy import Strategy
from random import randint
import pickle
from tqdm import tqdm

In [31]:
# load dataset
data = pd.read_csv("data/btcusdt24mo.csv")

In [32]:
# set first column as index
old_name = [name for name in data]
data = data.rename(columns={old_name[0]:'timestamp'})
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data = data.set_index(['timestamp'])

In [33]:
# rename columns
column_names = ['Open', 'High', 'Low', 'Close', 'Volume']
columns = {old_name[i+1]:column_name for i, column_name in enumerate(column_names)}
data = data.rename( columns=columns,)

In [34]:
# retain open, high, low, close and volume
data = data[column_names]

In [ ]:
# create EMA table
ema_size = 100
price = data.Close
ema_table = np.empty((len(price),ema_size), float)
ema_factor = np.linspace(.0001, 1.0, ema_size)
price_old = price[0]
for i in tqdm(range(len(price))):
    current_price = price[i]
    ema_table[i] = (1.0 - ema_factor)*price_old + ema_factor*current_price
    price_old = current_price

# to file
with open('ema_table', 'wb') as file:
    pickle.dump(ema_table, file)

In [35]:
ema_size = 100
with open('data/btcusdt24mo_ema_table', 'rb') as file:
    ema_table = pickle.load(file)

In [29]:
x = np.ones((1,ema_size+1), float)
x[0][:-1] = ema_table[0]
x[0][-1]

1.0

In [36]:
# ============================================== #
#               build startegy class             #
# ============================================== #

# set parameters
min_initial_samples = 20000
episode_len = randint(1000, 10000)
start_index = randint(min_initial_samples, len(data) - episode_len)
eval_data = data.iloc[start_index:start_index+episode_len]
eval_ema_table = ema_table[start_index:start_index+episode_len]
batch_size = 32
agent = Agent(ema_size + 1)

In [37]:
class train_model(Strategy):

    def init(self):
        self.index = 0
        self.set_data(eval_data.iloc[self.index]) # to get initial price for inititial position assesment
        self.state = self.get_state(ema=eval_ema_table[self.index])
        self.equity = self.get_equity(current_price=eval_data.iloc[self.index].Close)

    def next(self):
        action = agent.act(self.state) # planned action

        if action == 0 and self.get_current_position() == 1:
            action = 0 if self.sell() else 1 # get the actual action

        elif action == 1 and self.get_current_position() == 0:
            action = 1 if self.buy() else 0 # get the actual action

        next_state = self.get_state(ema=eval_ema_table[self.index + 1])
        next_equity = self.get_equity(current_price=eval_data.iloc[self.index + 1].Close)
        reward = (next_equity / self.equity) - 1.0
        agent.memory.append((self.state, action, reward, next_state))
        self.index += 1
        self.state = next_state
        self.equity = next_equity
        
        # if len(agent.memory) > batch_size:
        #     agent.expReplay(batch_size)

    def get_state(self, ema):
        ema = ema.copy()
        mean = np.mean(ema)
        std = np.std(ema)
        ema = (ema - mean) / std
        ema -= ema[0]
        state_array = np.empty((1, agent.state_size), float)
        state_array[0][:-1] = ema
        state_array[0][-1] = float(self.get_current_position())
        return state_array
    
    def get_current_position(self):
        current_cash_value = self.get_cash()
        current_coin_value = self.get_coins() * self.data.Close
        if current_cash_value > current_coin_value:
            return 0
        return 1
    
    def get_equity(self, current_price):
        return self.get_cash() + current_price * self.get_coins()

In [38]:
e = 1
while True:
    print('Episode {}:'.format(e))
    episode_len = randint(1000, 10000)
    start_index = randint(min_initial_samples, len(data) - episode_len)
    eval_data = data.iloc[start_index:start_index+episode_len]
    eval_ema_table = ema_table[start_index:start_index+episode_len]

    bt = Backtest(  data = eval_data.iloc[:-1],
                strategy = train_model,
                cash = 10000.0,
                coins = 0.0,
                commission = 0.001,
                tick_size = 0.01000000,
                lot_size = 0.00000100,)

    bt.run()

    agent.model.save("models/model_ep" + str(e))

    e += 1

Episode 1:
creating empty data frame...
100%|██████████| 3748/3748 [00:12<00:00, 311.48it/s, activity_log_time=0.96 ms, Equity=13.72% (1371.92 USDT)]
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/model_ep1\assets
Episode 2:
creating empty data frame...
100%|██████████| 2660/2660 [00:08<00:00, 313.48it/s, activity_log_time=1.12 ms, Equity=25.94% (2594.11 USDT)]
INFO:tensorflow:Assets written to: models/model_ep2\assets
Episode 3:
creating empty data frame...
100%|██████████| 2156/2156 [00:07<00:00, 279.88it/s, activity_log_time=0.81 ms, Equity=32.12% (3211.84 USDT)]
INFO:tensorflow:Assets written to: models/model_ep3\assets
Episode 4:
creating empty data frame...
100%|██████████| 3975/3975 [00:11<00:00, 331.42it/s, activity_log_time=0.72 ms, Equity=14.02% (1402

KeyboardInterrupt: 

In [ ]:
x = train_model()

In [ ]:
x._coins = 0

In [ ]:
x._cash = 1000

In [ ]:
x.set_data(eval_data.iloc[0])
x.get_state(ema=eval_ema_table[0])


In [ ]:
state = x.get_state(ema=eval_ema_table[0])

In [ ]:
state.shape

In [ ]:
agent.model.predict(np.reshape(state, (1, 101)))

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
m = Sequential()

In [ ]:
m.add(Dense(units=64, input_dim=3, activation="relu"))
m.add(Dense(units=32, activation="relu"))
m.add(Dense(units=8, activation="relu"))
m.add(Dense(2, activation="linear"))
m.compile(loss="mse", optimizer=Adam(lr=0.001))

In [ ]:
m.summary()

In [ ]:
m.predict(np.reshape(q,(1,3)))

In [ ]:
q = np.array([1,2,3])


In [ ]:
state.shape

In [ ]:
np.reshape(q,(1,3))